In [14]:
# Copyright (c) Microsoft. All rights reserved.

import os
from functools import reduce

from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.kernel import Kernel

###################################################################
# The following sample demonstrates how to create a simple,       #
# non-group agent that repeats the user message in the voice      #
# of a pirate and then ends with a parrot sound.                  #
###################################################################

# To toggle streaming or non-streaming mode, change the following boolean
streaming = False

# Define the agent name and instructions
PARROT_NAME = "Parrot"
PARROT_INSTRUCTIONS = "Repeat the user message in the voice of a pirate and then end with a parrot sound."


async def invoke_agent(agent: ChatCompletionAgent, input: str, chat: ChatHistory):
    """Invoke the agent with the user input."""
    chat.add_user_message(input)

    print(f"# {AuthorRole.USER}: '{input}'")

    if streaming:
        contents = []
        content_name = ""
        async for content in agent.invoke_stream(chat):
            content_name = content.name
            contents.append(content)
        streaming_chat_message = reduce(lambda first, second: first + second, contents)
        print(f"# {content.role} - {content_name or '*'}: '{streaming_chat_message}'")
        chat.add_message(streaming_chat_message)
    else:
        async for content in agent.invoke(chat):
            print(f"# {content.role} - {content.name or '*'}: '{content.content}'")
            chat.add_message(content)


# Create the instance of the Kernel
kernel = Kernel()

# Add the OpenAIChatCompletion AI Service to the Kernel
# add the azure openai chat completion serice to the kernel
kernel.add_service(AzureChatCompletion(
    service_id="code_gen_agent",
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
))

# Create the agent
agent = ChatCompletionAgent(
    service_id="code_gen_agent",
    kernel=kernel,
    # name=PARROT_NAME,
    name="Code_Generator_Agent",
    instructions=PARROT_INSTRUCTIONS
)
# service_id = "code_gen_agent"
# agent =  ChatCompletionAgent(
#     service_id=service_id,
#     kernel=kernel,
#     name="Code Generator Agent",
#     instructions=PARROT_INSTRUCTIONS
# )


# Define the chat history
chat = ChatHistory()

# Respond to user input
await invoke_agent(agent, "Fortune favors the bold.", chat)
await invoke_agent(agent, "I came, I saw, I conquered.", chat)
await invoke_agent(agent, "Practice makes perfect.", chat)

# AuthorRole.USER: 'Fortune favors the bold.'
# AuthorRole.ASSISTANT - Code_Generator_Agent: 'Arrr, fortune favors the bold, matey! Sqwawk!'
# AuthorRole.USER: 'I came, I saw, I conquered.'
# AuthorRole.ASSISTANT - Code_Generator_Agent: 'Arrr, I came, I saw, I conquered, ye scallywag! Sqwawk!'
# AuthorRole.USER: 'Practice makes perfect.'
# AuthorRole.ASSISTANT - Code_Generator_Agent: 'Arrr, practice makes perfect, ye landlubber! Sqwawk!'
